# Prediction on Test Set

In [1]:
# Clone GitHub Repository
!git clone https://github.com/sabin74/Enterprise-Intelligent-Demand-Forecasting-Decision-Optimization-Platform.git

Cloning into 'Enterprise-Intelligent-Demand-Forecasting-Decision-Optimization-Platform'...
remote: Enumerating objects: 372, done.
remote: Counting objects: 100% (156/156), done.
remote: Compressing objects: 100% (146/146), done.
remote: Total 372 (delta 86), reused 46 (delta 10), pack-reused 216 (from 1)
Receiving objects: 100% (372/372), 42.16 MiB | 29.96 MiB/s, done.
Resolving deltas: 100% (198/198), done.
Filtering content: 100% (26/26), 367.45 MiB | 48.61 MiB/s, done.


In [2]:
!pip install -q catboost
!pip install category_encoders


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.9/85.9 kB 3.0 MB/s eta 0:00:00


## Load Test Data

In [14]:

# Environment Setup - Import Libraries
import os
import gc
import joblib
import numpy as np
import pandas as pd
from pathlib import Path
from matplotlib import pyplot as plt

import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostRegressor
from category_encoders import TargetEncoder

import warnings
warnings.filterwarnings("ignore")

In [4]:
# Set Project Root
os.chdir("/content/Enterprise-Intelligent-Demand-Forecasting-Decision-Optimization-Platform")
print("Current Directory: ", os.getcwd())

Current Directory:  /content/Enterprise-Intelligent-Demand-Forecasting-Decision-Optimization-Platform


In [16]:
# Load Feature-Engineered Data
DATA_DIR = Path("data/features")

train = pd.read_parquet(DATA_DIR / "train_features.parquet")
test  = pd.read_parquet(DATA_DIR / "test_features.parquet")

print("Train shape:", train.shape)
print("Test shape :", test.shape)

# Sort for time-series
train = train.sort_values(["store_nbr", "family", "date"]).reset_index(drop=True)
test  = test.sort_values(["store_nbr", "family", "date"]).reset_index(drop=True)

gc.collect()

Train shape: (3054348, 62)
Test shape : (28512, 38)


0

<class 'pandas.core.frame.DataFrame'>
Index: 28512 entries, 0 to 28379
Data columns (total 38 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   id                   28512 non-null  int64         
 1   date                 28512 non-null  datetime64[ns]
 2   store_nbr            28512 non-null  int64         
 3   family               28512 non-null  category      
 4   onpromotion          28512 non-null  int64         
 5   city                 28512 non-null  category      
 6   state                28512 non-null  category      
 7   store_type           28512 non-null  category      
 8   cluster              28512 non-null  int64         
 9   dcoilwtico           28512 non-null  float64       
 10  holiday_type         28512 non-null  category      
 11  locale               28512 non-null  category      
 12  locale_name          28512 non-null  category      
 13  description          28512 non-null 

In [ ]:
# Memory Optimization (reduce memory usage)
def reduce_mem_usage(df, ):
  for col in df.columns:
    if df[col].dtype == "float64":
      df[col] = df[col].astype("float32")
    elif df[col].dtype == "int64":
      df[col] = df[col].astype("int32")
  return df

train = reduce_mem_usage(train)
gc.collect()


0